# Rode wijn in Portugal

Voor de Data Science casusopdracht heeft onze groep de rode wijn data set als onderzoekopdracht. In het eerste gedeelte van dit document zal er een een verkenning van de data plaatsvinden, waardoor er gerichte en interessante onderzoeksvragen opgesteld kunnen worden.

## Verkenning

Als eerste stap van de verkenning zal er onderzoek gedaan worden naar de huidige velden en de mogelijke verbeteringen aan de data set.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
redwine = pd.read_csv('redwine.csv', sep=';')
redwine.head()

In [ ]:
redwine.set_index('id', inplace=True)

In [ ]:
# Eerst eens bekijken hoeveel records we op dit moment hebben
len(redwine)

In [ ]:
# Bekijken hoeveel records het scheelt met ontbrekende waardes
len(redwine.dropna())

Hier zit best een fors verschil tussen, waardoor er met meer detail naar de data gekeken moet worden.

Daarnaast valt het ons op dat de country en variety velden allemaal gelijke data bevatten, waardoor deze irrelevant zijn.

In [ ]:
redwine.drop(['country', 'variety'], axis=1, inplace=True)
redwine.head()

Hieronder tellen we het aantal rijen met missende data per kolom.

In [ ]:
redwine.isna().sum()

Het valt op dat alle missende data voorkomt bij designation en price, echter zijn er ook een aantal kolommen die niet het goede type zijn. Deze zullen zeer waarschijnlijk dus ook data bevatten die niet als float gecast kan worden.

In [ ]:
redwine.dtypes

In [ ]:
redwine[np.isnan(pd.to_numeric(redwine["density"], errors="coerce"))]

Hieruit blijkt dat er maar 1 rij is waar de density niet klopt, omdat het er maar 1 is, zullen we deze verwijderen uit de dataset.

In [ ]:
redwine.drop(1475, inplace=True)
redwine["density"] = redwine["density"].astype("float")

De andere velden met het type object bevatten meer invalide waardes:

In [ ]:
print(len(redwine[np.isnan(pd.to_numeric(redwine["citric acid"], errors="coerce"))]))
print(len(redwine[np.isnan(pd.to_numeric(redwine["alcohol"], errors="coerce"))]))

Omdat het bij alcohol ook om maar een aantal waardes gaat, is er gekozen om ook voor de records met ongeldige alcohol waardes de records te verwijderen.

In [ ]:
redwine = redwine[~np.isnan(pd.to_numeric(redwine["alcohol"], errors="coerce"))]
redwine["alcohol"] = redwine["alcohol"].astype("float")

### Tweede dataset

Als tweede dataset is er een set gepakt welke ook de zelfde chemische samenstellingen bevat als de huidige set, alleen is deze gericht op witte wijnen. 

In [ ]:
whitewine = pd.read_csv('winequality-white.csv', sep=';')
whitewine.head()

In tegenstelling tot bij de rode wijn, is er niet veel informatie over de regio en naamgeving van de wijn beschikbaar. Daarnaast is het een kwaliteitscijfer en niet een waardering van een reviewer, waardoor de cijfers niet direct te vergelijken zijn.

Bij de witte wijn set worden alle kolommen wel correct ingelezen:

In [ ]:
whitewine.dtypes


#### Histogrammen

Om meer inzicht te krijgen in de verdelingen van de stoffen, is voor elke stof een histogram opegsteld. Ook voor de pH, quality en alcohol waardes zijn grafieken gemaakt.

In [ ]:
sns.set()
for column in whitewine.columns.values:
    plt.title(column)
    if column in redwine:
        if redwine[column].dtype != "object":
            plt.hist(redwine[column], color="#ff335577")
    plt.hist(whitewine[column], color="#ffff3377")
    plt.show()

Het valt hierbij op dat de quality en citric acid geen rode wijnen heeft, omdat de quality niet aanwezig is als veld in de dataset, en omdat citric acid van het type object is. In tegenstelling tot alcohol en density is hier nog geen oplossing voor, omdat er veel meer datapunten missen.

Daarnaast is het interessant om te zien dat veel van deze grafieken geen goeie normaalverdeling vormen, wat doet vermoeden dat er een directe aanleiding is voor de verdeling van de stoffen. De grafiek van de pH waardes lijkt nog het meest normaal verdeeld.

## Onderzoeksvragen

Een eis voor het voltooien van deze casus is het opstellen (en beantwoorden) van onderzoeksvragen. Hierbij moet er rekening gehouden worden met de data die is aangeleverd en moet er een combinatie gemaakt worden met een externe set van data.

In het vorige gedeelte is een externe set verkend, welke betrekking heeft op witte wijnen in plaats van rode.

Met deze twee datasets en alle gegevens erin zijn we tot de volgende onderzoeksvragen gekomen:

- Welke proefpersonen geven gemiddeld meer of minder punten voor een wijn en hoeveel is dit?
- Zijn er groepen te onderscheiden in de wijn op basis van de stoffen en de wijnmakerijen?
- In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?
- Wat zijn de verschillen in verhoudingen van de chemische kenmerken bij rode t.o.v. witte wijnen?


## Onderzoeksvraag 1

Deze onderzoeksvraag luidt: "Welke proefpersonen geven gemiddeld meer of minder punten voor een wijn en hoeveel is dit?"

Door deze vraag te beantwoorden kan er gekeken worden of alle wijnen even eerlijk beoordeeld worden. Mocht er dan een vergelijking gaan komen tussen de verschillende wijnen waarbij het aantal punten een rol speelt kan er rekening mee gehouden worden.

Als eerste zullen we bekijken hoeveel reviewers er in totaal zijn en wat hun gemiddelde scores zijn:

In [ ]:
redwine['taster_name'].astype(str)
redwine_grouped = redwine.groupby(['taster_name'])[['taster_name', 'points']]
display(redwine_grouped.mean())

Hierboven zijn de gemiddelden per reviewer te zien.
Hierin valt op dat de gemiddelden allemaal binnen 5 punten van elkaar liggen.
Een verschil van een gemiddelde van 5 op een totaal van zo te zien 100 punten is niet genoeg om te zeggen dat er één of meerdere personen daadwerkelijk een andere waardering geven ten opzichte van de rest. Hieruit is voor deze deelvraag dus nog niks af te leiden.

Om er toch dieper op in te duiken zal er hieronder een histogram worden opgesteld voor elk proefpersoon. Hiermee kunnen we kijken of er ook echt uitschieters gegeven worden. Eventueel is er dan verder mee te onderzoeken of de lage(re) of hoge(re) scores zich allemaal bij bepaalde wijnen bevinden.

In [ ]:
plt.hist(redwine['points'], label='Uitgedeelde scores')
plt.legend()
plt.show()

Hierboven is te zien dat, zoals ook al af te leiden uit de gemiddelden hierboven, de meeste scores tussen de 85 en 90 zijn gegeven. 
Er ontstaat hier dan ook een mooie normaalverdeling in de uitgegedeelde punten.

In [ ]:
display(redwine_grouped.size())

Hierboven is per reviewer te zien hoeveel punten er daadwerkelijk uitgedeeld zijn. Hieruit is dus af te leiden of en persoon ook genoeg scores heeft uigedeeld heeft om betrouwbaar een gemiddelde vanaf te leiden.

Daarnaast is het interessant om te bekijken of de personen met een minder aantal uitgedeelde punten ook een hogere spreiding hebben in hun uitgedeelde punten.

In [ ]:
for taster_name, data in redwine_grouped:
    if data['points'].count() < 20:
        plt.hist(data['points'], label=taster_name)
        plt.legend()
        plt.show()

Hierboven is in de histogrammen te zien dat de meeste spreiding plaats vind bij de volgende personen:
- Carrie Dykes
- Fiona Adams
- Mike DeSimone

Ook valt het op dat alledrie de personen 5 of minder reviews hebben gegeven.

Wanneer we het dan verder gaan ontleden en dit afzetten ten opzichte van de wijnen die ze gewaardeerd hebben zien we het volgende voor 'Carrie Dykes' (beginnend bij de laagste score):

In [ ]:
display(redwine[redwine['taster_name'] == 'Carrie Dykes'].sort_values(['points']))

Nu is dus te zien dat de wijn genaamd 'Casal da Coelheira 2011 Red (Tejo)' de wijn is welke het minste gewaardeerd wordt door Carrie Dykes met 83 punten. Als er dan gekeken word naar alle scores gegeven aan deze wijn zien we de volgende data:

In [ ]:
display(redwine[redwine['title'] == 'Casal da Coelheira 2011 Red (Tejo)'].sort_values(['points']))

Deze wijn is geen goede peiler om te bekijken aangezien deze niet door een ander persoon gewaardeerd is. Om te voorkomen dat dit bij alles voorkomt word er bekeken of voor alle wijnen geld dat deze maar door een enkel persoon getest is.

In [ ]:
(redwine['title'] == 'Parras Vinhos 2010 Montaria Red (Alentejano)').sum()

Bij de eerstvolgende wijn beoordeeld door Carrie Dykes is al te zien dat er 2 personen deze wijn beoordeeld hebben. Met deze informatie is het dus al duidelijk dat er meerdere personen zijn die dezelfde wijn beoordelen.

Hieronder volgen de overige twee wijnen beoordeeld door Carrie:

In [ ]:
display(redwine[redwine['title'] == 'Parras Vinhos 2010 Montaria Red (Alentejano)'].sort_values(['points']))

Hier is te zien dat Carrie Dykes exact dezelfde score heeft gegeven op dezelfde wijn als 'Roger Voss' welke dan weer meer als 600 reviews op zijn naam heeft staan. Hier word er dus uitgegaan van een betrouwbare score.

Als volgende word de derde beoordeelde door Carrie Dykes behandeld.

In [ ]:
display(redwine[redwine['title'] == 'J, Portugal Ramos 2013 Ramos Reserva Red (Alentejano)'].sort_values(['points']))

Hier is helaas duidelijk dat er maar 1 persoon is geweest welke een score heeft gegeven aan deze wijn.
Voor nu is de conclusie dus dat de scores gegeven door deze persoon **onbetrouwbaar** zijn omdat op minder als de helft van de gegeven scores een referentie punt gevonden kan worden in andere reviewers.

---

Als volgende worden de wijnen van 'Fiona Adams' behandeld.

In [ ]:
display(redwine[redwine['taster_name'] == 'Fiona Adams'].sort_values(['points']))

Ook voor Fiona Adams zijn er weer 3 wijnen welke onderzocht zullen worden. Wat ook te zien is, is dat Fiona Adams in vergelijking met Carrie Dykes dus wel hogere punten uitdeeld.

Beginnende met de eerste wijn genaamd 'Quinta do Cavalinho':

In [ ]:
display(redwine[redwine['title'] == 'Quinta do Cavalinho 2013 Vale das Donas Red (Tejo)'].sort_values(['points']))

Helaas is ook hier weer Fiona Adams de enige persoon die een score heeft gegeven aan de wijn.

Door naar de volgende:

In [ ]:
display(redwine[redwine['title'] == 'Quinta do Casal Branco 2009 Quartilho Red (Ribatejo)'].sort_values(['points']))

Ook hier is het helaas weer alleen Fiona Adams die als enigste een score heeft uitgegeven.

De derde wijn als volgt:

In [ ]:
display(redwine[redwine['title'] == 'Barão de Vilar 2014 Cais da Ribeira Reserva Red (Douro)'].sort_values(['points']))

Bij de derde zien we dan wel dat Fiona Adams hetzelfde aantal punten heeft gegeven als 'Michael Schachner'.
Helaas komt het totaal er net als bij Carrie Dykes eropneer dat er **niet** voldoende data is om deze persoon als **betrouwbaar** aan te kunnen merken.

---

Dan als laatste word 'Mike DeSimone' behandeld:

In [ ]:
display(redwine[redwine['taster_name'] == 'Mike DeSimone'].sort_values(['points']))

Deze persoon heeft in totaal 5 wijnen beoordeeld.

beginnende met de wijn 'Sogevinus 2006 Curva Tinto Red (Douro)':

In [ ]:
display(redwine[redwine['title'] == 'Sogevinus 2006 Curva Tinto Red (Douro)'].sort_values(['points']))

Ondanks dat deze prima beoordeeld is op een schaal van 0 tot 100 is dit weer door 1 persoon gebeurd.

De volgende te onderzoeken is 'Companhia das Quintas 2011 Pancas Red (Lisboa)':

In [ ]:
display(redwine[redwine['title'] == 'Companhia das Quintas 2011 Pancas Red (Lisboa)'].sort_values(['points']))

Deze is wel door een andere beoordeeld. Dit is dezelfde 'Roger Voss' welke eerder ook al voorkwam bij 'Carrie Dykes' hierdoor kunnen we dus zeggen dat deze betrouwbaar is.

In [ ]:
display(redwine[redwine['title'] == 'Quinta de la Rosa 2011 Red (Douro)'].sort_values(['points']))

Deze wijn is weer door Mike DeSimone alleen beoordeeld.

In [ ]:
display(redwine[redwine['title'] == 'Real Companhia Velha 2013 Evel Reserva Red (Douro)'].sort_values(['points']))

Hierbij hetzelfde als de vorige, alleen door Mike DeSimone beoordeeld.

In [ ]:
display(redwine[redwine['title'] == 'Casa da Passarella 2012 Abanico Reserva Red (Dão)'].sort_values(['points']))

Hier hetzelfde.
Bij de persoon 'Mike DeSimone' is de conclusie dus ook **onbetrouwbaar** op basis van het te weinig kunnen peilen van de beoordelingen.

### Conlusie onderzoeksvraag 1
Als conclusie kunnen we dus zeggen dat de personen welke een minder "normale" normaalverdeling hadden in hun scores, door het te weinig geven van beoordelingen om deze goed te kunnen peilen, ook te onbetrouwbaar zijn om een duidelijk antwoord op te geven.

Wel is het zo dat de scores die vergeleken konden worden, gelijkwaardig zijn aan de scores gegeven door de meer ervaren personen.
Als conclusie valt hier dus uit op te maken, uitgaande dat de andere wijnen gelijkwaardig beoordeeld zouden worden, dat er geen personen zijn die een erg afwijkende score geven of een score welke eventueel bijgesteld zou kunnen worden.

## Onderzoeksvraag 2

Voor de tweede onderzoeksvraag zullen er groepen worden gemaakt op basis van de stoffen en de wijnmakerijen. Deze vraag is een clustering onderzoek, waardoor er bij voorbaad geen richtlijnen opgesteld kunnen worden voor een volledige beantwoording van de onderzoeksvraag. 

In [ ]:
from sklearn.cluster import KMeans
whitewine_clustering = whitewine

### Experimenten

Allereerst zullen er een aantal experimenten volgen rondom het clusteren van de witte wijn dataset op basis van verschillende eigenschappen.
Hierbij zullen er geen definitieve conclusies getrokken worden, maar ligt de nadruk op het vinden van interessante uitkomsten.
Wel is er soms extra toelichting over wat er te zien is in de verschillende plots.
De uitwerkingen in de clustering paragraaf zijn voorzien van correcte labels en een titel, de diagrammen hieronder dus niet.

In [ ]:
means = KMeans(n_clusters=3, random_state=0)
means.fit(whitewine_clustering[["alcohol", "sulphates"]])
means.cluster_centers_

In [ ]:
plt.scatter(whitewine_clustering["alcohol"], whitewine_clustering["sulphates"], c=means.labels_)

In dit cluster van drie groepen zijn er duidelijk 3 groepen gemaakt op basis van het alcohol percentage.

In [ ]:
means = KMeans(n_clusters=6, random_state=0)
means.fit(whitewine_clustering[["alcohol", "sulphates"]])
plt.scatter(whitewine_clustering["alcohol"], whitewine_clustering["sulphates"], c=means.labels_)

Wederom lijken de groepen enkel gebaseerd te zijn op het alcohol percentage.

In [ ]:
means = KMeans(n_clusters=4, random_state=0)
means.fit(whitewine_clustering[["alcohol", "quality"]])
plt.scatter(whitewine_clustering["alcohol"], whitewine_clustering["quality"], c=means.labels_)

In [ ]:
means = KMeans(n_clusters=4, random_state=0)
means.fit(whitewine_clustering[["pH", "alcohol"]])
plt.scatter(whitewine_clustering["pH"], whitewine_clustering["alcohol"], c=means.labels_)

In [ ]:
means = KMeans(n_clusters=4, random_state=0)
means.fit(whitewine_clustering[["alcohol", "residual sugar"]])
plt.scatter(whitewine_clustering["alcohol"], whitewine_clustering["residual sugar"], c=means.labels_)

In [ ]:
means = KMeans(n_clusters=4, random_state=0)
means.fit(whitewine_clustering[["pH", "residual sugar"]])
plt.scatter(whitewine_clustering["pH"], whitewine_clustering["residual sugar"], c=means.labels_)

In [ ]:
whitewine_clustering.groupby(means.labels_).describe()

Uit latere tests is gebleken dat er uitschieters in de data zitten, deze worden hieronder weggehaald uit de dataset:

In [ ]:
redwine_clustering = redwine[["points", "fixed acidity", "volatile acidity", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "pH", "sulphates", "alcohol"]]

whitewine_clustering = whitewine_clustering[whitewine_clustering["total sulfur dioxide"] < 200]
redwine_clustering = redwine_clustering[redwine_clustering["total sulfur dioxide"] < 250]

### Clustering

Hieronder zal het belangrijkste gedeelte van de clustering plaatsvinden, waarbij er gezocht zal worden naar een aantal groepen tussen de 3 en 5.
Voor elk van deze clustering uitkomsten zal worden onderzocht of er logische groepen ontstaan en of deze te herleiden zijn naar bepaalde eigenschappen. Dit zal zowel voor de rode als de witte wijn data set uitgevoerd worden.

In [ ]:
three_group_white_cluster = KMeans(n_clusters=3, random_state=0)
three_group_white_cluster.fit(whitewine_clustering)
whitewine_clustering.groupby(three_group_white_cluster.labels_).mean()

Hierbij valt op dat er voornamelijk verschillen tussen de groepen zitten voor de residual sugar, free sulfur dioxide, total sulfur dioxide en alchohol.

In [ ]:
four_group_white_cluster = KMeans(n_clusters=4, random_state=0)
four_group_white_cluster.fit(whitewine_clustering)
whitewine_clustering.groupby(four_group_white_cluster.labels_).mean()

Bij het clusteren in vier groepen valt op dat de grote verschillen wederom bij deze eigenschappen te vinden zijn.

In [ ]:
five_group_white_cluster = KMeans(n_clusters=5, random_state=0)
five_group_white_cluster.fit(whitewine_clustering)
whitewine_clustering.groupby(five_group_white_cluster.labels_).mean()

Bij vijf groepen is dit niet anders. Dat de grootste verschillen zitten in deze velden, blijkt voor het 2e, 3e en 4e genoemde element zelfs hetzelfde te zijn voor de rode wijn:

In [ ]:
three_group_red_cluster = KMeans(n_clusters=3, random_state=0)
three_group_red_cluster.fit(redwine_clustering)
redwine_clustering.groupby(three_group_red_cluster.labels_).mean()

In [ ]:
four_group_red_cluster = KMeans(n_clusters=4, random_state=0)
four_group_red_cluster.fit(redwine_clustering)
redwine_clustering.groupby(four_group_red_cluster.labels_).mean()

In [ ]:
five_group_red_cluster = KMeans(n_clusters=5, random_state=0)
five_group_red_cluster.fit(redwine_clustering)
redwine_clustering.groupby(five_group_red_cluster.labels_).mean()

Hieruit blijkt dat KMeans voor deze wijn datasets voornamelijk het onderscheid maakt op basis van de sulfur dioxide, en daarnaast gedeeltelijk op basis van alcohol en residual sugar. Om te visualiseren hoe groot de verschillen tussen de groepen zijn, zullen hieronder verschillende diagrammen volgen.

In [ ]:
plt.title("3 groups white wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(whitewine_clustering["free sulfur dioxide"], whitewine_clustering["total sulfur dioxide"], c=three_group_white_cluster.labels_)

In [ ]:
plt.title("3 groups red wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(redwine_clustering["free sulfur dioxide"], redwine_clustering["total sulfur dioxide"], c=three_group_red_cluster.labels_)

Het is hierbij leuk om te vermelden dat er redelijk vergelijkbare figuren en groepen voortkomen uit beide datasets. Hieronder zijn beide plots weergegeven met dezelfde assen:

In [ ]:
plt.title("3 groups white wine clustering with the same axes")
plt.ylim(0, 200)
plt.xlim(0, 100)
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(whitewine_clustering["free sulfur dioxide"], whitewine_clustering["total sulfur dioxide"], c=three_group_white_cluster.labels_)

In [ ]:
plt.title("3 groups red wine clustering with the same axes")
plt.ylim(0, 200)
plt.xlim(0, 100)
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(redwine_clustering["free sulfur dioxide"], redwine_clustering["total sulfur dioxide"], c=three_group_red_cluster.labels_)

Door deze grafieken met elkaar te vergelijken, is duidelijk te concluderen dat rode wijn over het algemeen veel minder sulfur dioxide bevat. Dit blijkt ook uit de gemiddelde waardes voor beide tabellen:

In [ ]:
redwine_clustering.mean()

In [ ]:
whitewine_clustering.mean()

In [ ]:
plt.title("5 groups white wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(whitewine_clustering["free sulfur dioxide"], whitewine_clustering["total sulfur dioxide"], c=five_group_white_cluster.labels_)

In [ ]:
plt.title("5 groups red wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(redwine_clustering["free sulfur dioxide"], redwine_clustering["total sulfur dioxide"], c=five_group_red_cluster.labels_)

Ook met 5 groepen valt duidelijk te zien dat de verdeling voornamelijk gebaseerd is op de totale hoeveelheid sulfur dioxide. Dit roept meteen de vraag op waarom er dan een zichtbaar verschil zat tussen de alcohol percentages. Om deze vraag te beantwoorden zal er gekeken worden naar de verhouding en relatie tussen sulfur dioxide en alcohol.

### Alcohol vs. sulfur dioxide

In dit laatste onderdeel van de tweede deelvraag zal er onderzoek gedaan worden naar de relatie van deze stoffen, omdat dit de stoffen waarom die de meeste invloed hadden op de groepsverdeling.

In [ ]:
plt.title("group 0 red wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(redwine_clustering[three_group_red_cluster.labels_ == 0]["alcohol"])

In [ ]:
plt.title("group 1 red wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(redwine_clustering[three_group_red_cluster.labels_ == 1]["alcohol"])

In [ ]:
plt.title("group 2 red wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(redwine_clustering[three_group_red_cluster.labels_ == 2]["alcohol"])

In de bovenstaande grafieken is duidelijk te zien dat er in de groep 0 een veel groter aandeel van sterkere rode wijnen is. Ditzelfde is ook te zien bij de witte wijnen, waar groep 0 aanzienlijk minder sterkere wijnen bevat:

In [ ]:
plt.title("group 0 white wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(whitewine_clustering[three_group_white_cluster.labels_ == 0]["alcohol"])

In [ ]:
plt.title("group 1 white wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(whitewine_clustering[three_group_white_cluster.labels_ == 1]["alcohol"])

In [ ]:
plt.title("group 2 white wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(whitewine_clustering[three_group_white_cluster.labels_ == 2]["alcohol"])

In [ ]:
whitewine_clustering.groupby(three_group_white_cluster.labels_)["alcohol"].mean()

In [ ]:
plt.title("white wine clustering")
plt.xlabel("total sulfur dioxide")
plt.ylabel("alcohol")
plt.scatter(whitewine_clustering["total sulfur dioxide"], whitewine_clustering["alcohol"], c=five_group_white_cluster.labels_)

In [ ]:
plt.title("red wine clustering")
plt.xlabel("total sulfur dioxide")
plt.ylabel("alcohol")
plt.scatter(redwine_clustering["total sulfur dioxide"], redwine_clustering["alcohol"], c=five_group_red_cluster.labels_)

De bovenstaande plots zijn interessant, want hierin is niet alleen nogmaals te zien dat witte wijn veel meer sulfur dioxide bevat, maar ook dat sterke rode wijnen vaak weinig sulfur dioxide bevatten, terwijl dit bij witte wijnen heel anders werkt.

In [ ]:
sns.jointplot(whitewine_clustering["total sulfur dioxide"], whitewine_clustering["alcohol"], kind="reg")

In [ ]:
sns.jointplot(redwine_clustering["total sulfur dioxide"], redwine_clustering["alcohol"], kind="reg")

Voor de meeste wijnen is wel te zeggen dat een hoge hoeveelheid alcohol in verband staat met een lage hoeveelheid sulfur dioxide.
Dit is niet het geval voor de hoeveelheid free sulfur dioxide, zoals hieronder te zien is:

In [ ]:
plt.title("white wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("alcohol")
plt.scatter(whitewine_clustering["free sulfur dioxide"], whitewine_clustering["alcohol"], c=five_group_white_cluster.labels_)

In [ ]:
plt.title("red wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("alcohol")
plt.scatter(redwine_clustering["free sulfur dioxide"], redwine_clustering["alcohol"], c=five_group_red_cluster.labels_)

Daarnaast valt het op de de groepen veel minder goed te onderscheiden zijn in deze plots, omdat de groepen dus voornamelijk gebaseerd zijn op de totale hoeveelheid sulfur dioxide.
Ook is er een verband tussen een hoge hoeveelheid alcohol en een lage hoeveelheid totaal sulfur dioxide.

### Koppeling met wijnmakerijen

In dit onderdeel van de onderzoeksvraag zal de terugkoppeling van de clusters naar de wijnmakerijen uitgevoerd worden.
Het is voor te stellen dat de clusters ook gedeeltelijk overeen komen met de wijnmakerijen, maar het kan ook zijn dat hier geen relatie in te ontdekken is. De wijnmakerij is alleen bekend bij de rode wijn dataset.

In [ ]:
redwine_original_with_no_outliers = redwine[redwine["total sulfur dioxide"] < 250]
groups = redwine_original_with_no_outliers.groupby(five_group_red_cluster.labels_)["winery"]
groups.describe()

In [ ]:
for group in range(0, 5):
    print(f"CLUSTER {group}")
    print(groups.value_counts()[group].head())

Hier zijn de wijnmakerijen matig te relateren aan de groepen, waarbij het opvalt dat "Wines & Winemakers" en "DFJ Vinhos" vaak verschuiven, ondanks dat zij veel wijnen in elke categorie hebben.

### Correlatie overzichten

In deze laatste paragraaf zijn nog correlatie overzichten te zien voor de stoffen.

In [ ]:
redwine_original_with_no_outliers.corr()

In [ ]:
whitewine_clustering.corr()

Hier valt te zien dat voor de rode wijnen de prijs en alcohol matig correleren met waardering van de wijn (points). Ook valt te concluderen dat er wel een verband is tussen total sulfur dioxide en alcohol, maar dat het niet om correlatie gaat (-0.264086).

### Conclusie

Als conclusie op het clusteren van de wijnen op basis van de stoffen en andere chemische eigenschappen zou er gezegd kunnen worden dat rode wijnen vaak minder sulfur dioxide bevatten.
Daarnaast is er een verband tussen enkele stoffen te ontdekken, waarop het kmeans algoritme de groepen indeeld.

Vervolgens is er nog onderzoek gedaan naar de relatie van de clusters met de wijnmakerijen. Hier zijn een aantal verbanden in te vinden, voornamelijk de verhouding van wijnen per cluster van "Wines & Winemakers" t.o.v. "DFJ Vinhos" is interessant.

Als laatste blijkt dat er correlatie tussen de prijs en alcohol met de waardering is.

Hiermee is onderzoeksvraag 2 beantwoord.